# Pivoting 

For this example, let's look at two datasets. One a voting data from here: https://www.kaggle.com/stevepalley/2016uspresidentialvotebycounty

The other, world happiness data from here: https://www.kaggle.com/unsdsn/world-happiness

This dataset will make use of our merging skills!

In [1]:
import pandas as pd
import numpy as np

dfo = pd.read_csv("pres16results.csv")
dfo.head()

county fips             cand  st  pct_report       votes  total_votes  \
0    NaN   US     Donald Trump  US      0.9951  60350241.0  127592176.0   
1    NaN   US  Hillary Clinton  US      0.9951  60981118.0  127592176.0   
2    NaN   US     Gary Johnson  US      0.9951   4164589.0  127592176.0   
3    NaN   US       Jill Stein  US      0.9951   1255968.0  127592176.0   
4    NaN   US    Evan McMullin  US      0.9951    451636.0  127592176.0   

        pct          lead  
0  0.472993  Donald Trump  
1  0.477938  Donald Trump  
2  0.032640  Donald Trump  
3  0.009844  Donald Trump  
4  0.003540  Donald Trump

In [2]:
top = dfo.loc[dfo.fips == "US", ["cand", "votes"]].sort_values("votes", ascending=False)
top.head()

cand       votes
1  Hillary Clinton  60981118.0
0     Donald Trump  60350241.0
2     Gary Johnson   4164589.0
3       Jill Stein   1255968.0
4    Evan McMullin    451636.0

In [3]:
candidates = top.cand.head()
candidates

1    Hillary Clinton
0       Donald Trump
2       Gary Johnson
3         Jill Stein
4      Evan McMullin
Name: cand, dtype: object

In [4]:
df = dfo[dfo.cand.isin(candidates)]
df.head()

county fips             cand  st  pct_report       votes  total_votes  \
0    NaN   US     Donald Trump  US      0.9951  60350241.0  127592176.0   
1    NaN   US  Hillary Clinton  US      0.9951  60981118.0  127592176.0   
2    NaN   US     Gary Johnson  US      0.9951   4164589.0  127592176.0   
3    NaN   US       Jill Stein  US      0.9951   1255968.0  127592176.0   
4    NaN   US    Evan McMullin  US      0.9951    451636.0  127592176.0   

        pct          lead  
0  0.472993  Donald Trump  
1  0.477938  Donald Trump  
2  0.032640  Donald Trump  
3  0.009844  Donald Trump  
4  0.003540  Donald Trump

## Pivot

In [5]:
p = df.pivot(index="fips", columns="cand", values="votes")
p

cand   Donald Trump  Evan McMullin  Gary Johnson  Hillary Clinton  Jill Stein
fips                                                                         
10001       36989.0            NaN        2876.0          33347.0      1041.0
10003       85507.0            NaN        9101.0         162905.0      3955.0
10005       62607.0            NaN        2774.0          39329.0      1104.0
1001        18110.0            NaN         538.0           5908.0       105.0
1003        72780.0            NaN        2448.0          18409.0       453.0
...             ...            ...           ...              ...         ...
VT          95053.0            NaN       10047.0         178179.0      6748.0
WA        1043648.0            NaN      131779.0        1523720.0     46016.0
WI        1409467.0            NaN      106442.0        1382210.0     30980.0
WV         486198.0            NaN       22798.0         187457.0      8000.0
WY         174248.0            NaN       13285.0          55949.0      2512.0

[3165 rows x 5 columns]

In [6]:
df.set_index(["fips", "cand"]).votes.unstack()

cand   Donald Trump  Evan McMullin  Gary Johnson  Hillary Clinton  Jill Stein
fips                                                                         
10001       36989.0            NaN        2876.0          33347.0      1041.0
10003       85507.0            NaN        9101.0         162905.0      3955.0
10005       62607.0            NaN        2774.0          39329.0      1104.0
1001        18110.0            NaN         538.0           5908.0       105.0
1003        72780.0            NaN        2448.0          18409.0       453.0
...             ...            ...           ...              ...         ...
VT          95053.0            NaN       10047.0         178179.0      6748.0
WA        1043648.0            NaN      131779.0        1523720.0     46016.0
WI        1409467.0            NaN      106442.0        1382210.0     30980.0
WV         486198.0            NaN       22798.0         187457.0      8000.0
WY         174248.0            NaN       13285.0          55949.0      2512.0

[3165 rows x 5 columns]

Election data is boring, but its served its purpose well. 

## Pivot_table

Is a generalised version of pivot. Handles duplicate values, generates MultiIndexes for you. This is the one I'd normally use.

Now lets look at world happiness from here:

https://www.kaggle.com/unsdsn/world-happiness

Alas this is split into files by year, so lets merge them all together.

In [7]:
data = [pd.read_csv(f"{year}.csv").assign(Year=year) for year in range(2015, 2020)]
for d in data:
    print(d.Year[0], d.columns)

2015 Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual', 'Year'],
      dtype='object')
2016 Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual', 'Year'],
      dtype='object')
2017 Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual', 'Year'],
      dtype='object')
2018 Index(['Overall rank', 'Country or region', 'Score', 'GDP per capita',
       'Social supp

In [8]:
renames = {
    "Country or region": "Country",
    "Happiness Score": "Score",
    "Happiness.Score": "Score",
    "Economy..GDP.per.Capita": "GDP per capita",
    "Economy..GDP.per.Capita.": "GDP per capita",
    "Economy (GDP per Capita)": "GDP per capita",
}
keep = ["Country", "Score", "Year", "GDP per capita"]
data_prepped = [d.rename(columns=renames).loc[:, keep] for d in data]

In [9]:
df = pd.concat(data_prepped, sort=False).reset_index(drop=True)

In [10]:
with pd.option_context('display.max_rows', 800, 'display.max_columns', 30):
    display(df)

Country  Score  Year  GDP per capita
0                 Switzerland  7.587  2015        1.396510
1                     Iceland  7.561  2015        1.302320
2                     Denmark  7.527  2015        1.325480
3                      Norway  7.522  2015        1.459000
4                      Canada  7.427  2015        1.326290
5                     Finland  7.406  2015        1.290250
6                 Netherlands  7.378  2015        1.329440
7                      Sweden  7.364  2015        1.331710
8                 New Zealand  7.286  2015        1.250180
9                   Australia  7.284  2015        1.333580
10                     Israel  7.278  2015        1.228570
11                 Costa Rica  7.226  2015        0.955780
12                    Austria  7.200  2015        1.337230
13                     Mexico  7.187  2015        1.020540
14              United States  7.119  2015        1.394510
15                     Brazil  6.983  2015        0.981240
16                 Luxembourg  6.946  2015        1.563910
17                    Ireland  6.940  2015        1.335960
18                    Belgium  6.937  2015        1.307820
19       United Arab Emirates  6.901  2015        1.427270
20             United Kingdom  6.867  2015        1.266370
21                       Oman  6.853  2015        1.360110
22                  Venezuela  6.810  2015        1.044240
23                  Singapore  6.798  2015        1.521860
24                     Panama  6.786  2015        1.063530
25                    Germany  6.750  2015        1.327920
26                      Chile  6.670  2015        1.107150
27                      Qatar  6.611  2015        1.690420
28                     France  6.575  2015        1.277780
29                  Argentina  6.574  2015        1.053510
30             Czech Republic  6.505  2015        1.178980
31                    Uruguay  6.485  2015        1.061660
32                   Colombia  6.477  2015        0.918610
33                   Thailand  6.455  2015        0.966900
34               Saudi Arabia  6.411  2015        1.395410
35                      Spain  6.329  2015        1.230110
36                      Malta  6.302  2015        1.207400
37                     Taiwan  6.298  2015        1.290980
38                     Kuwait  6.295  2015        1.554220
39                   Suriname  6.269  2015        0.995340
40        Trinidad and Tobago  6.168  2015        1.211830
41                El Salvador  6.130  2015        0.764540
42                  Guatemala  6.123  2015        0.745530
43                 Uzbekistan  6.003  2015        0.632440
44                   Slovakia  5.995  2015        1.168910
45                      Japan  5.987  2015        1.270740
46                South Korea  5.984  2015        1.244610
47                    Ecuador  5.975  2015        0.864020
48                    Bahrain  5.960  2015        1.323760
49                      Italy  5.948  2015        1.251140
50                    Bolivia  5.890  2015        0.681330
51                    Moldova  5.889  2015        0.594480
52                   Paraguay  5.878  2015        0.759850
53                 Kazakhstan  5.855  2015        1.122540
54                   Slovenia  5.848  2015        1.184980
55                  Lithuania  5.833  2015        1.147230
56                  Nicaragua  5.828  2015        0.593250
57                       Peru  5.824  2015        0.900190
58                    Belarus  5.813  2015        1.031920
59                     Poland  5.791  2015        1.125550
60                   Malaysia  5.770  2015        1.124860
61                    Croatia  5.759  2015        1.082540
62                      Libya  5.754  2015        1.131450
63                     Russia  5.716  2015        1.137640
64                    Jamaica  5.709  2015        0.810380
65               North Cyprus  5.695  2015        1.208060
66                     Cyprus  5.689  2015        1.208130
67        

In [11]:
df.to_csv("happiness.csv", index=False, float_format="%0.3f")

In [12]:
df.pivot_table(index="Country", values="Score")

Score
Country             
Afghanistan  3.51280
Albania      4.71260
Algeria      5.66760
Angola       3.87225
Argentina    6.45940
...              ...
Venezuela    5.53140
Vietnam      5.15460
Yemen        3.62580
Zambia       4.58440
Zimbabwe     4.00660

[170 rows x 1 columns]

In [13]:
df.groupby("Country").Score.mean().to_frame()

Score
Country             
Afghanistan  3.51280
Albania      4.71260
Algeria      5.66760
Angola       3.87225
Argentina    6.45940
...              ...
Venezuela    5.53140
Vietnam      5.15460
Yemen        3.62580
Zambia       4.58440
Zimbabwe     4.00660

[170 rows x 1 columns]

In [15]:
df.pivot_table(index="Country", values="Score", aggfunc=[np.mean, np.std, np.median])

mean       std  median
               Score     Score   Score
Afghanistan  3.51280  0.232634  3.5750
Albania      4.71260  0.145607  4.6550
Algeria      5.66760  0.465170  5.6050
Angola       3.87225  0.112272  3.8305
Argentina    6.45940  0.231017  6.5740
...              ...       ...     ...
Venezuela    5.53140  0.897944  5.2500
Vietnam      5.15460  0.123009  5.1030
Yemen        3.62580  0.294979  3.5930
Zambia       4.58440  0.392690  4.5140
Zimbabwe     4.00660  0.397866  3.8750

[170 rows x 3 columns]

In [18]:
df.pivot_table(index="Country", columns="Year")

GDP per capita                                   Score         \
Year                  2015     2016      2017   2018   2019   2015   2016   
Country                                                                     
Afghanistan        0.31982  0.38227  0.401477  0.332  0.350  3.575  3.360   
Albania            0.87867  0.95530  0.996193  0.916  0.947  4.959  4.655   
Algeria            0.93929  1.05266  1.091864  0.979  1.002  5.605  6.355   
Angola             0.75778  0.84731  0.858428  0.730    NaN  4.033  3.866   
Argentina          1.05351  1.15137  1.185295  1.073  1.092  6.574  6.650   
...                    ...      ...       ...    ...    ...    ...    ...   
Venezuela          1.04424  1.13367  1.128431  0.996  0.960  6.810  6.084   
Vietnam            0.63216  0.74037  0.788548  0.715  0.741  5.360  5.061   
Yemen              0.54649  0.57939  0.591683  0.442  0.287  4.077  3.724   
Zambia             0.47038  0.61202  0.636407  0.562  0.578  5.129  4.795   
Zimbabwe           0.27100  0.35041  0.375847  0.357  0.366  4.610  4.193   

                                  
Year          2017   2018   2019  
Country                           
Afghanistan  3.794  3.632  3.203  
Albania      4.644  4.586  4.719  
Algeria      5.872  5.295  5.211  
Angola       3.795  3.795    NaN  
Argentina    6.599  6.388  6.086  
...            ...    ...    ...  
Venezuela    5.250  4.806  4.707  
Vietnam      5.074  5.103  5.175  
Yemen        3.593  3.355  3.380  
Zambia       4.514  4.377  4.107  
Zimbabwe     3.875  3.692  3.663  

[170 rows x 10 columns]

In [20]:
df.pivot_table(index="Country", columns="Year").swaplevel(0, 1, axis=1).sort_index(axis=1, level=0)

Year                  2015                  2016                  2017         \
            GDP per capita  Score GDP per capita  Score GDP per capita  Score   
Country                                                                         
Afghanistan        0.31982  3.575        0.38227  3.360       0.401477  3.794   
Albania            0.87867  4.959        0.95530  4.655       0.996193  4.644   
Algeria            0.93929  5.605        1.05266  6.355       1.091864  5.872   
Angola             0.75778  4.033        0.84731  3.866       0.858428  3.795   
Argentina          1.05351  6.574        1.15137  6.650       1.185295  6.599   
...                    ...    ...            ...    ...            ...    ...   
Venezuela          1.04424  6.810        1.13367  6.084       1.128431  5.250   
Vietnam            0.63216  5.360        0.74037  5.061       0.788548  5.074   
Yemen              0.54649  4.077        0.57939  3.724       0.591683  3.593   
Zambia             0.47038  5.129        0.61202  4.795       0.636407  4.514   
Zimbabwe           0.27100  4.610        0.35041  4.193       0.375847  3.875   

Year                  2018                  2019         
            GDP per capita  Score GDP per capita  Score  
Country                                                  
Afghanistan          0.332  3.632          0.350  3.203  
Albania              0.916  4.586          0.947  4.719  
Algeria              0.979  5.295          1.002  5.211  
Angola               0.730  3.795            NaN    NaN  
Argentina            1.073  6.388          1.092  6.086  
...                    ...    ...            ...    ...  
Venezuela            0.996  4.806          0.960  4.707  
Vietnam              0.715  5.103          0.741  5.175  
Yemen                0.442  3.355          0.287  3.380  
Zambia               0.562  4.377          0.578  4.107  
Zimbabwe             0.357  3.692          0.366  3.663  

[170 rows x 10 columns]

### Recap
* pivot
* pivot_table
* comparison to groupby